In [ ]:
#python version 3.11.7
import pkg_resources
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from boruta import BorutaPy 
import optuna 
import joblib 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
import pickle 
from statannot import add_stat_annotation 
import shap

#np.random.seed(42)

#for boruta
np.int = np.int32
np.float = np.float64
np.bool = np.bool_


In [ ]:
data_dir = 'dataset/'

In [ ]:

UCEC_full = pd.read_csv("dataset/TCGA_UCEC_scaled.csv",sep=',', index_col=0)
UCEC_full = UCEC_full[UCEC_full.columns.difference(['RNA_count'])]
UCEC_full = UCEC_full.dropna(how = 'any')
UCEC_full.columns

In [ ]:
# The column where each feature is located may vary, so you'll need to manually adjust it

# UCEC_ARID1A: TCPA protein expression column
UCEC_ARID1A = UCEC_full.iloc[:,0]
UCEC_RNA = UCEC_full.iloc[:,68]
UCEC_Mut = UCEC_full.iloc[:,[2,3,4,6]]
UCEC_CNV = UCEC_full.iloc[:,1]
UCEC_Met = UCEC_full.iloc[:,7:35]
UCEC_miRNA = UCEC_full.iloc[:,35:68]
UCEC_Biogrid = pd.read_csv("/dataset/Biogrid_feature.csv",sep=',', index_col=0)
UCEC_Biogrid = UCEC_Biogrid.loc[UCEC_full.index,:]
UCEC_KEGG = pd.read_csv("/dataset/KEGG_feature.csv",sep=',', index_col=0)
UCEC_KEGG = UCEC_KEGG.loc[UCEC_full.index,:]


In [ ]:
#Result_matrix = pd.DataFrame(columns=['train_RMSE','train_R2','train_R','test_RMSE','test_R2','test_R'])
#Make result matrix for each random seed 
UCEC_data_list = ['RNA','CNV','miRNA','Met','Mut','KEGG','Biogrid'] #7
Models = ['LinearRegression','Ridge','SVR','ElasticNet','RandomForestRegressor','XGBRegressor','LGBMRegressor'] #7


In [ ]:

#Linear Regression
linear = LinearRegression(n_jobs= -1)

# Define SVM Regressor
def SVR_objective(trial):
    params = { 
        #kernel funciton
        'kernel' : trial.suggest_categorical('kernel',['linear', 'poly', 'rbf', 'sigmoid']),
        # how far influences the calculation of plausible line of separation
        'gamma': trial.suggest_float('gamma', 1e-5, 1, log=True),
        # C : regularisation, how much error is bearable, higher = overfitting 
        'C': trial.suggest_float('C', 1e-5, 1, log=True)
    }
    
    model = SVR(**params)
    svr_cv = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= -1)
    return np.mean(svr_cv)

# Define Ridge Regressor
def Ridge_objective(trial):
    params = { 
        # alpha = regularization strength
        'alpha' : trial.suggest_float('alpha', 0.1, 100, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
    }
    model = Ridge(**params)
    ridge_cv = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= -1)
    return np.mean(ridge_cv)

# Define ElasticNet Regressor
def ElasticNet_objective(trial):
    params = { 
        # alpha = regularization strength
        'alpha' : trial.suggest_float('alpha', 0.1, 100, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'l1_ratio' : trial.suggest_float('l1_ratio', 0.01, 1.0, log=True),
    }
    model = ElasticNet(**params)
    elastic_cv = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= -1)
    return np.mean(elastic_cv)

# Define Randomforest Regressor
def RandomForestRegressor_objective(trial):
    params = { 
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000, step=100),
        'max_depth': trial.suggest_int('max_depth', 5, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap',[True, False]),
        'n_jobs' : -1,
    }
    model = RandomForestRegressor(**params)
    rf_cv = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= -1)
    return np.mean(rf_cv)

# Define XGBoost Regressor
def XGBRegressor_objective(trial):
    params = {
        'eval_metric' : 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        # use exact for small dataset.
        "tree_method": "exact",
        "eta": trial.suggest_float("eta",1e-2,0.1,log = True),
        # L2 regularization weight.
        "reg_lambda": trial.suggest_float('reg_lambda', 1e-3, 10.0),
        # L1 regularization weight.
        "reg_alpha": trial.suggest_float('reg_alpha', 1e-3, 10.0),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.6,1,step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 0.9, step=0.1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.2, 0.9, step=0.1),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        "max_depth" : trial.suggest_int("max_depth", 1, 9),
        'min_child_weight' :  trial.suggest_int("min_child_weight", 2, 10),
        'n_jobs' : -1,
    }
    model = XGBRegressor(**params)
    xg_cv = -1 * cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= -1)
    return np.mean(xg_cv)

# Define LGBM Regressor 
def LGBMRegressor_objective(trial):
    params = {
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log = True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.6,1,step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1, step=0.1),
        "max_depth" : trial.suggest_int("max_depth", 1, 9),
        'min_child_weight' :  trial.suggest_int("min_child_weight", 2, 10),
        'early_stopping_round': 100,
        'n_jobs' : -1,
        'verbose' : -100
    }
    X_cv, X_eval, y_cv, y_eval = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    fit_params = {
    'eval_set': [(X_eval, y_eval)]
    }
    model = LGBMRegressor(**params)
    lgb_cv = -1 * cross_val_score(model, X_cv, y_cv, cv=5, scoring='neg_mean_squared_error', n_jobs= -1,
                                  fit_params=fit_params, verbose = 0)
    return np.mean(lgb_cv)



In [ ]:
#for loop for 10 different random seed
for i in range(10):
    globals()['Result_matrix_'+ str(i)] = pd.DataFrame(columns=['train_RMSE','train_R2','train_R','test_RMSE','test_R2','test_R'])
    np.random.seed(i)
    for level in UCEC_data_list:
        X = globals()['UCEC_' + level]
        y = UCEC_ARID1A
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        if (level in ['KO','RNA','CNV','Biogrid']):
            X_train = np.array(X_train).reshape(-1, 1)
            X_test = np.array(X_test).reshape(-1, 1)
        for model in Models:
            if model == 'LinearRegression':
                cur_model = LinearRegression(n_jobs= -1)
            else:
                study = optuna.create_study(direction='minimize')
                study.optimize(globals()[model+'_objective'], n_trials=100) #10 : test trial 
                best_params = study.best_params
                cur_model = globals()[model](**best_params)
            cur_model.fit(X_train, y_train)
            train_y_pred = cur_model.predict(X_train)
            train_rmse = np.sqrt(mean_squared_error(y_train, train_y_pred))
            train_r2 = r2_score(y_train, train_y_pred)
            train_corr = np.corrcoef(y_train, train_y_pred)[0,1]
            test_y_pred = cur_model.predict(X_test)
            test_rmse = np.sqrt(mean_squared_error(y_test, test_y_pred))
            test_r2 = r2_score(y_test, test_y_pred)
            test_corr = np.corrcoef(y_test, test_y_pred)[0,1]
            globals()['Result_matrix_'+ str(i)].loc[level+'_'+model] = [train_rmse,train_r2,train_corr,test_rmse,test_r2,test_corr]
    globals()['Result_matrix_'+ str(i)].to_csv("Result_matrix/Result_matrix_"+str(i)+".csv")

## RESULT Analysis

In [ ]:
Result_matrix_0 = pd.read_csv("Result_matrix/Result_matrix_0.csv", index_col=0)
Result_matrix_1 = pd.read_csv("Result_matrix/Result_matrix_1.csv", index_col=0)
Result_matrix_2 = pd.read_csv("Result_matrix/Result_matrix_2.csv", index_col=0)
Result_matrix_3 = pd.read_csv("Result_matrix/Result_matrix_3.csv", index_col=0)
Result_matrix_4 = pd.read_csv("Result_matrix/Result_matrix_4.csv", index_col=0)
Result_matrix_5 = pd.read_csv("Result_matrix/Result_matrix_5.csv", index_col=0)
Result_matrix_6 = pd.read_csv("Result_matrix/Result_matrix_6.csv", index_col=0)
Result_matrix_7 = pd.read_csv("Result_matrix/Result_matrix_7.csv", index_col=0)
Result_matrix_8 = pd.read_csv("Result_matrix/Result_matrix_8.csv", index_col=0)
Result_matrix_9 = pd.read_csv("Result_matrix/Result_matrix_9.csv", index_col=0)


In [ ]:
dfs = list([Result_matrix_0,Result_matrix_1,Result_matrix_2,Result_matrix_3,Result_matrix_4,
           Result_matrix_5,Result_matrix_6,Result_matrix_7,Result_matrix_8,Result_matrix_9])

In [ ]:
concatenated_Result_matrix = pd.concat(dfs, axis = 1)


In [ ]:
rank_df = concatenated_Result_matrix.rank(axis=0, ascending= False)


In [ ]:
#average rank of test_R2
rank_df['test_R2'].mean(axis =1 ).sort_values()

In [ ]:
#average rank of test_R2
rank_df['test_RMSE'].mean(axis =1 ).sort_values(ascending= False)

In [ ]:
#average rank of test_R2
rank_df['test_R'].mean(axis =1).sort_values()

In [ ]:
### Ranking grouped by Omics level 

omics_Result_matrix = pd.DataFrame(concatenated_Result_matrix)
omics_Result_matrix.index = concatenated_Result_matrix.index.str.split('_').str[0]
omics_Result_matrix = omics_Result_matrix.groupby(level = 0).mean()


In [ ]:
avg_omics_Result_matrix = pd.DataFrame(index=UCEC_data_list)
avg_omics_Result_matrix['train_RMSE'] = omics_Result_matrix['train_RMSE'].mean(axis = 1)
avg_omics_Result_matrix['train_R2'] = omics_Result_matrix['train_R2'].mean(axis = 1)
avg_omics_Result_matrix['train_R'] = omics_Result_matrix['train_R'].mean(axis = 1)
avg_omics_Result_matrix['test_RMSE'] = omics_Result_matrix['test_RMSE'].mean(axis = 1)
avg_omics_Result_matrix['test_R2'] = omics_Result_matrix['test_R2'].mean(axis = 1)
avg_omics_Result_matrix['test_R'] = omics_Result_matrix['test_R'].mean(axis = 1)


In [ ]:
### Ranking grouped by Algorithm 
algorithm_Result_matrix = pd.DataFrame(concatenated_Result_matrix)
algorithm_Result_matrix.index = concatenated_Result_matrix.index.str.split('_').str[1]
algorithm_Result_matrix = algorithm_Result_matrix.groupby(level = 0).mean()


In [ ]:
avg_algorithm_Result_matrix = pd.DataFrame(index=Models)
avg_algorithm_Result_matrix['train_RMSE'] = algorithm_Result_matrix['train_RMSE'].mean(axis = 1)
avg_algorithm_Result_matrix['train_R2'] = algorithm_Result_matrix['train_R2'].mean(axis = 1)
avg_algorithm_Result_matrix['train_R'] = algorithm_Result_matrix['train_R'].mean(axis = 1)
avg_algorithm_Result_matrix['test_RMSE'] = algorithm_Result_matrix['test_RMSE'].mean(axis = 1)
avg_algorithm_Result_matrix['test_R2'] = algorithm_Result_matrix['test_R2'].mean(axis = 1)
avg_algorithm_Result_matrix['test_R'] = algorithm_Result_matrix['test_R'].mean(axis = 1)


In [ ]:
sns.barplot(avg_algorithm_Result_matrix, x = avg_algorithm_Result_matrix.index, y = "test_R2",
            order = avg_algorithm_Result_matrix.sort_values('test_R2').index)
plt.xticks(rotation = 45,ha = 'right')
plt.title("R-Squared")
plt.tight_layout()


In [ ]:
sns.barplot(avg_algorithm_Result_matrix, x = avg_algorithm_Result_matrix.index, y = "test_R",
            order = avg_algorithm_Result_matrix.sort_values('test_R').index)
plt.xticks(rotation = 45,ha = 'right')
plt.title("R")
plt.tight_layout()


In [ ]:
sns.barplot(avg_algorithm_Result_matrix, x = avg_algorithm_Result_matrix.index, y = "test_RMSE",
            order = avg_algorithm_Result_matrix.sort_values('test_RMSE').index)
plt.xticks(rotation = 45,ha = 'right')
plt.title("RMSE")
plt.tight_layout()


In [ ]:
sns.barplot(avg_omics_Result_matrix, x = avg_omics_Result_matrix.index, y = "test_RMSE",
            order = avg_omics_Result_matrix.sort_values('test_RMSE').index)
plt.xticks(rotation = 45)
plt.title("Omics level : test_RMSE")
plt.tight_layout()


In [ ]:
sns.barplot(avg_omics_Result_matrix, x = avg_omics_Result_matrix.index, y = "test_R2")
plt.xticks(rotation = 45)
plt.title("Omics level : test_R2")
plt.tight_layout()

In [ ]:
sns.barplot(avg_omics_Result_matrix, x = avg_omics_Result_matrix.index, y = "test_R")
plt.xticks(rotation = 45)
plt.title("Omics level : test_R")
plt.tight_layout()